# Writing Scripts for Tangible Landscape

***Caitlin Haedrich, Pratikshya Regmi, Anna Petrasova and Helena Mitasova***

*Center for Geospatial Analytics at NC State University*

<img src="./img/applications.jpg" />

[Tangible Landscape](https://tangible-landscape.github.io/) is a tangible user interface for GRASS, available as an addon on [GitHub](https://github.com/tangible-landscape). Using a physical landscape model for inputs, custom workflows can be easily constructed in a Python script then the results projected back onto the landscape.

Example workflows, referred to as activites in the documentation, can be found on GitHub and documentation on how to install, configure and build your own setup with custom activities is also on GitHub.

In this notebook, we'll convert 

***

## Imports and Start GRASS

Import the Python standard libraries we need.

In [ ]:
import subprocess
import sys
from pathlib import Path

sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj

In [ ]:
gj.init("./csdms-grass-2025/PERMANENT");

In [ ]:
!g.list type=all

In [ ]:
!g.region -p



---



## From Notebook Workflow to Executable Script

Tangible Landscape uses scripts to execute analyses on the scanned terrain. Here we show some examples of how to structure GRASS Python scripts.

The `%%writefile` cell magic takes the content of the cell and writes it to a file. The `%%python` magic will execute the file.

In [ ]:
%%file curvature.py
#!/usr/bin/env python3

import os
import grass.script as gs


def run_slope(scanned_elev, env, **kwargs):
    gs.run_command("r.slope.aspect", elevation=scanned_elev, slope="slope", env=env)


def main():
    env = os.environ.copy()
    env["GRASS_OVERWRITE"] = "1"
    elevation = "elev_lid792_1m"
    elev_resampled = "elev_resampled"
    gs.run_command("g.region", raster=elevation, res=4, flags="a", env=env)
    gs.run_command("r.resamp.stats", input=elevation, output=elev_resampled, env=env)

    run_slope(scanned_elev=elev_resampled, env=env)


if __name__ == "__main__":
    main()


Now test the script:

In [ ]:
%%python curvature.py

And visualize the result using the `grass.jupyter` API:

In [ ]:
map = gj.Map()
map.d_rast(map="pcurvature")
map.show()

TODO: add viewshed here from the coordinates so we can use it on TL

Now if your workflow requires a vector points map or coordinates, use this template:

In [ ]:
%%writefile yourlastname.py
import grass.script as gs

def get_coordinates(points):
    """Helper function to get coordinate pairs from a vector point layer.
    Do not modify."""
    data = gs.read_command("v.out.ascii", input=points, separator="comma").splitlines()
    return [[float(coor) for coor in point.split(",")[:2]] for point in data]

# modify here
# change function name
def myanalysis(elevation, points):
    """Traces a flow through an elevation model"""
    coordinates = get_coordinates(points)
    if coordinates:
        gs.run_command("r.drain", input=elevation, output="drain", drain="drain_vector", start_coordinates=coordinates)

if __name__ == "__main__":
    elevation = "elevation"
    points = "lagoon_points"
    myanalysis(elevation=elevation, points=points)

In [ ]:
%%python yourlastname.py

In [ ]:
map = gj.Map()
map.d_rast(map="elevation")
map.d_vect(map="drain_vector")
map.show()

## Activity JSON file